In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
data = pd.read_csv("/content/Original_Dataset.csv")
data.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Initialize dictionary to store symptoms for each disease
symptoms_dict = {}

# Iterate through each row of the DataFrame
for index, row in data.iterrows():
    # Initialize empty set to store symptoms for the current disease
    current_symptoms = set()
    disease = row['Disease']

    # Iterate through each symptom column of the row
    for col in row.index[1:]:
        symptom = row[col]

        # Check for NaN value
        if pd.isna(symptom):
            break  # Break the loop if NaN is encountered

        # Add symptom to current symptoms set
        current_symptoms.add(symptom)

    # Update symptoms for the current disease in the dictionary
    if disease in symptoms_dict:
        symptoms_dict[disease].update(current_symptoms)
    else:
        symptoms_dict[disease] = current_symptoms

# Convert dictionary to list of lists
symptoms = [list(symptoms) for disease, symptoms in symptoms_dict.items()]
diseases = [disease for disease, symptoms in symptoms_dict.items()]
symptoms_cleaned = [
    [symptom.replace('_', ' ') if isinstance(symptom, str) else symptom for symptom in symptom_list]
    for symptom_list in symptoms
]

print(symptoms_cleaned)

[[' dischromic  patches', ' nodal skin eruptions', ' skin rash', 'itching'], [' watering from eyes', ' shivering', ' chills', ' continuous sneezing'], [' vomiting', ' acidity', ' ulcers on tongue', ' stomach pain', ' cough', ' chest pain'], [' abdominal pain', ' vomiting', ' yellowing of eyes', ' yellowish skin', ' loss of appetite', ' nausea', 'itching'], [' skin rash', ' spotting  urination', ' stomach pain', ' burning micturition', 'itching'], [' abdominal pain', ' vomiting', ' internal itching', ' indigestion', ' loss of appetite', ' passage of gases'], [' muscle wasting', ' patches in throat', ' high fever', ' extra marital contacts'], [' weight loss', ' excessive hunger', ' fatigue', ' irregular sugar level', ' increased appetite', ' blurred and distorted vision', ' obesity', ' polyuria', ' lethargy', ' restlessness'], [' vomiting', ' dehydration', ' sunken eyes', ' diarrhoea'], [' family history', ' fatigue', ' mucoid sputum', ' cough', ' breathlessness', ' high fever'], [' loss

In [4]:
len(symptoms_cleaned)

41

In [5]:
symptoms_strings = [' '.join(map(str, symptom_list)) for symptom_list in symptoms]

print(symptoms_strings)

[' dischromic _patches  nodal_skin_eruptions  skin_rash itching', ' watering_from_eyes  shivering  chills  continuous_sneezing', ' vomiting  acidity  ulcers_on_tongue  stomach_pain  cough  chest_pain', ' abdominal_pain  vomiting  yellowing_of_eyes  yellowish_skin  loss_of_appetite  nausea itching', ' skin_rash  spotting_ urination  stomach_pain  burning_micturition itching', ' abdominal_pain  vomiting  internal_itching  indigestion  loss_of_appetite  passage_of_gases', ' muscle_wasting  patches_in_throat  high_fever  extra_marital_contacts', ' weight_loss  excessive_hunger  fatigue  irregular_sugar_level  increased_appetite  blurred_and_distorted_vision  obesity  polyuria  lethargy  restlessness', ' vomiting  dehydration  sunken_eyes  diarrhoea', ' family_history  fatigue  mucoid_sputum  cough  breathlessness  high_fever', ' loss_of_balance  lack_of_concentration  dizziness  headache  chest_pain', ' excessive_hunger  acidity  indigestion  irritability  visual_disturbances  stiff_neck  

In [6]:
symptoms_list = [disease for disease, symptoms in symptoms_dict.items()]
len(symptoms_list)

41

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, GlobalAveragePooling1D, MultiHeadAttention
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Tokenization and padding
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(symptoms_strings)
sequences = tokenizer.texts_to_sequences(symptoms_strings)
padded_sequences = pad_sequences(sequences, padding='post')

# Hyperparameters
MAX_NUM_WORDS = len(tokenizer.word_index) + 1
MAX_SEQUENCE_LENGTH = padded_sequences.shape[1]
EMBEDDING_DIM = 50
NUM_CLASSES = len(set(diseases))
BATCH_SIZE = 1  # Set batch size to 1
EPOCHS = 100
DROPOUT_RATE = 0.5

# Transformer architecture
input_layer = Input(shape=(MAX_SEQUENCE_LENGTH,))
embedding_layer = Embedding(input_dim=MAX_NUM_WORDS, output_dim=EMBEDDING_DIM)(input_layer)
attention_layer = MultiHeadAttention(num_heads=8, key_dim=EMBEDDING_DIM)(embedding_layer, embedding_layer)
global_avg_pooling_layer = GlobalAveragePooling1D()(attention_layer)
dropout_layer = Dropout(DROPOUT_RATE)(global_avg_pooling_layer)
output_layer = Dense(NUM_CLASSES, activation='softmax')(dropout_layer)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Convert diseases to integer labels
disease_to_label = {disease: idx for idx, disease in enumerate(set(diseases))}
labels = [disease_to_label[disease] for disease in diseases]

# Training the model
history = model.fit(padded_sequences, np.array(labels), batch_size=BATCH_SIZE, epochs=EPOCHS)

# Make predictions
def predict_disease(symptoms_list):
    cleaned_symptoms = [symptom.replace('_', ' ') if isinstance(symptom, str) else symptom for symptom in symptoms_list]
    cleaned_symptoms = [' '.join(map(str, symptom_list)) for symptom_list in cleaned_symptoms]
    sequences = tokenizer.texts_to_sequences(cleaned_symptoms)
    padded_sequences = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
    predictions = model.predict(padded_sequences)
    predicted_label_idx = np.argmax(predictions, axis=1)[0]
    predicted_disease = list(disease_to_label.keys())[list(disease_to_label.values()).index(predicted_label_idx)]
    return predicted_disease


Epoch 1/100
41/41 [==============================] - 6s 63ms/step - loss: 3.7477 - accuracy: 0.0000e+00
Epoch 2/100
41/41 [==============================] - 0s 4ms/step - loss: 3.7246 - accuracy: 0.0244
Epoch 3/100
41/41 [==============================] - 0s 4ms/step - loss: 3.7204 - accuracy: 0.0244
Epoch 4/100
41/41 [==============================] - 0s 5ms/step - loss: 3.6963 - accuracy: 0.0488
Epoch 5/100
41/41 [==============================] - 0s 5ms/step - loss: 3.6608 - accuracy: 0.0732
Epoch 6/100
41/41 [==============================] - 0s 4ms/step - loss: 3.5463 - accuracy: 0.0732
Epoch 7/100
41/41 [==============================] - 0s 4ms/step - loss: 3.4429 - accuracy: 0.0488
Epoch 8/100
41/41 [==============================] - 0s 4ms/step - loss: 3.3574 - accuracy: 0.0244
Epoch 9/100
41/41 [==============================] - 0s 4ms/step - loss: 3.1826 - accuracy: 0.0732
Epoch 10/100
41/41 [==============================] - 0s 5ms/step - loss: 3.2380 - accuracy: 0.0976
Epoc

In [8]:
disease_to_label

{'Hepatitis D': 0,
 'Drug Reaction': 1,
 'Impetigo': 2,
 'Tuberculosis': 3,
 'Acne': 4,
 'Psoriasis': 5,
 'Migraine': 6,
 'Jaundice': 7,
 'GERD': 8,
 '(vertigo) Paroymsal  Positional Vertigo': 9,
 'Cervical spondylosis': 10,
 'Gastroenteritis': 11,
 'AIDS': 12,
 'hepatitis A': 13,
 'Urinary tract infection': 14,
 'Bronchial Asthma': 15,
 'Hypoglycemia': 16,
 'Common Cold': 17,
 'Alcoholic hepatitis': 18,
 'Diabetes ': 19,
 'Hepatitis B': 20,
 'Typhoid': 21,
 'Dimorphic hemmorhoids(piles)': 22,
 'Heart attack': 23,
 'Hypertension ': 24,
 'Chronic cholestasis': 25,
 'Fungal infection': 26,
 'Hepatitis C': 27,
 'Paralysis (brain hemorrhage)': 28,
 'Hyperthyroidism': 29,
 'Pneumonia': 30,
 'Dengue': 31,
 'Chicken pox': 32,
 'Peptic ulcer diseae': 33,
 'Hypothyroidism': 34,
 'Arthritis': 35,
 'Malaria': 36,
 'Osteoarthristis': 37,
 'Allergy': 38,
 'Hepatitis E': 39,
 'Varicose veins': 40}

In [9]:
MAX_SEQUENCE_LENGTH = padded_sequences.shape[1]
MAX_SEQUENCE_LENGTH

30

In [10]:
import pickle
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

In [ ]:
# Save model state dictionary
import torch
import pickle
with open("custom_transformer.pkl", "wb") as f:
    pickle.dump(model.to_json(), f)
    model.save_weights("custom_transformer_weights.h5")

In [ ]:
import pickle
import tensorflow as tf
from tensorflow.keras.models import model_from_json

# Load model architecture
with open("custom_transformer.pkl", "rb") as f:
    model_json = pickle.load(f)
    model = model_from_json(model_json)

# Load model weights
model.load_weights("custom_transformer_weights.h5")

In [ ]:
# Test prediction
test_symptoms = [' dehydration', ' blurred_and_distorted_vision', ' polyuria']
print(f"Predicted Disease: {predict_disease([test_symptoms])}")

1/1 [==============================] - 0s 195ms/step
Predicted Disease: Diabetes 


In [11]:
doc = pd.read_csv("/content/Doc_specialisation.csv", names=['Disease', 'Specialist'])
doc.head()

,Disease,Specialist
NaN,Doctor's Name,speciality
0.0,"Diem Truong, LAc, MSTOM",Acupuncturist
1.0,"Monique Rivera, LAc",Acupuncturist
2.0,"Ronald Pratt, LAc, DiplAc, MA, MSAc",Acupuncturist
3.0,"Daniel Camburn, LAc",Acupuncturist


In [12]:
dvd = pd.read_csv("/content/Doctor_Versus_Disease (1).csv", encoding='latin1', names=['Disease', 'Specialist'])
dvd.head()

,Disease,Specialist
0,Drug Reaction,Allergist
1,Allergy,Allergist
2,Hypertension,Cardiologist
3,Heart attack,Cardiologist
4,Psoriasis,Dermatologist


In [13]:
doc['Specialist'] = np.where((doc['Disease'] == 'Tuberculosis'), 'Pulmonologist', doc['Specialist'])
doc = doc.rename(columns={'speciality': 'Specialist'})
merged_df = dvd.merge(doc, on='Specialist', how='left')
merged_df = merged_df.rename(columns={'Disease_y': 'Name'})
merged_df

,Disease_x,Specialist,Name
0,Drug Reaction,Allergist,"Dr. Roshni R. Naik, MD"
1,Drug Reaction,Allergist,"Dr. Roshni R. Naik, MD"
2,Drug Reaction,Allergist,"Dr. Vahid Rahimian, DO"
3,Drug Reaction,Allergist,"Dr. Ira Finegold, MD, MS"
4,Drug Reaction,Allergist,"Dr. Neha Sirohi, MD"
...,...,...,...
3693,Pneumonia,Pulmonologist,"Dr. George Castro, MD"
3694,Pneumonia,Pulmonologist,"Dr. Pervaiz Iqbal, MD, FCCP"
3695,Osteoarthristis,Rheumatologists,NaN
3696,Arthritis,Rheumatologists,NaN


In [14]:
import pandas as pd

# Sample merged_df DataFrame
# merged_df = pd.DataFrame({
#     "disease_x": ["Heart disease", "Respiratory infection", "Migraine"],
#     "specialist": ["Cardiologist", "Pulmonologist", "Neurologist"],
#     "Name": ["Doctor A", "Doctor B", "Doctor C"]
# })

def predict_disease_and_doctor(symptoms_list, merged_df):
    predicted_disease = predict_disease([symptoms_list])
    # Fetch doctor's specialization
    doctor_specialization = dvd[dvd["Disease"] == predicted_disease]["Specialist"].values[0] # Default to General Physician if not found

    # Fetch doctor's name from merged_df based on specialization
    filtered_doctors = merged_df[merged_df["Specialist"] == doctor_specialization]["Name"].str.replace('\xa0', '').tolist()

    return predicted_disease, doctor_specialization, filtered_doctors


In [16]:
predicted_disease, doctor_specialization, filtered_doctors = predict_disease_and_doctor( [ " i have  and dehydration"], merged_df)
print("Predicted Disease:", predicted_disease)
print("Doctor's Specialization:", doctor_specialization)
print("Matching Doctors:", filtered_doctors)

1/1 [==============================] - 0s 40ms/step
Predicted Disease: Heart attack
Doctor's Specialization: Cardiologist
Matching Doctors: ['Dr.Sonal Jani,MD', 'Dr.Kenneth Giedd,MD', 'Dr.Azadeh Beheshtian,MD', 'Dr.Azadeh Beheshtian,MD', 'Dr.Pramod Sanghi,MD', 'Dr.Dmitry Gorelik,MD', 'Dr.Salvatore Cavaleri,MD', 'Dr.Abiola Dele-Michael,MD, FACC, MPH', 'Dr.Abiola Dele-Michael,MD, FACC, MPH', 'Dr.Peter Virzi,MD', 'Dr.Maurice Rachko,MD', 'Dr.Jeffrey Wessler,MD, MPH', 'Dr.Kenneth Giedd,MD', 'Dr.Peter Virzi,MD', 'Dr.Robert Kornberg,MD', 'Dr.Shyam Amin,MD', 'Dr.Shyam Amin,MD', 'Dr.Edo Paz,MD', 'Dr.Sonal Jani,MD', 'Dr.Sonal Jani,MD', 'Dr.Sonal Jani,MD', 'Dr.Marc Rybstein,MD', 'Dr.Pramod Sanghi,MD', 'Dr.Krysthel Engstrom,MD', 'Dr.Sonal Jani,MD', 'Dr.Jeffrey Kohn,MD, FACC', 'Dr.Mary Greene,MD', 'Dr.Alexander Mauskop,MD, FACC', 'Dr.Salvatore Cavaleri,MD', 'Dr.Robert Segal,MD', 'Dr.Alexander Mauskop,MD, FACC', 'Dr.Gargi Bajpayee,MD', 'Dr.Gargi Bajpayee,MD', 'Dr.Mark Meyer,MD, FACC, FACP', 'Dr.Amno

In [ ]:
# Save DataFrame to CSV file
merged_df.to_csv("merged_doctor_specialization.csv", index=False)

In [ ]:
with open("custom_transformer.json", "w") as json_file:
    json_file.write(model.to_json())